In [1]:
import spacy

In [2]:
spacy.__version__

'3.0.3'

In [3]:
nlp = spacy.load("en_core_web_trf")

In [4]:
nlp.pipe_names

['transformer', 'tagger', 'parser', 'ner', 'attribute_ruler', 'lemmatizer']

In [5]:
nlp.meta['name']

'core_web_trf'

In [6]:
#nlp("hello there")._.trf_data

In [7]:
from pathlib import Path

import os
import sys

current_path = Path(os.getcwd())
sys.path.append(str(current_path.parent))
os.environ['VECTORIAN_CPP_IMPORT'] = "1"

In [8]:
import vectorian

from vectorian.importers import NovelImporter
from vectorian.session import LabSession
from vectorian.embeddings import SpacyTransformerEmbedding

my_embedding = SpacyTransformerEmbedding(nlp).compressed(100)

# contextual embeddings
im = NovelImporter(nlp, embeddings=[my_embedding])

#im = NovelImporter(nlp)
doc = im("/Users/arbeit/Wise Children Mini.txt")

Matching checksum for /Users/arbeit/Projects/vectorian-2021/vectorian/core/cpp/core.cpp --> not compiling


Importing /Users/arbeit/Wise Children Mini.txt: 100%|██████████| 5/5 [00:02<00:00,  1.73it/s]


In [9]:
doc.contextual_embeddings

{'spacy/core_web_trf/3.0.0/pca:100': <vectorian.embeddings.ProxyVectorsRef at 0x7f83ec4312e0>}

In [10]:
doc.contextual_embeddings['spacy/core_web_trf/3.0.0/pca:100'].open().unmodified.shape

(837, 100)

In [9]:
doc.save("/Users/arbeit/Desktop/my_example_doc")

In [9]:
from vectorian.corpus.document import Document
doc = Document.load("/Users/arbeit/Desktop/my_example_doc")

In [9]:
# note: you may also specify StackedEmbedding here.
session = LabSession(
    [doc],
    embeddings=[my_embedding],
    normalizers="default")

?? 837


In [12]:
from vectorian.importers import StringImporter
from vectorian.session import LabSession
from vectorian.metrics import CosineSimilarity, TokenSimilarity
from vectorian.metrics import AlignmentSentenceSimilarity
from vectorian.alignment import WatermanSmithBeyer, ConstantGapCost

metric = AlignmentSentenceSimilarity(
    token_metric=TokenSimilarity(my_embedding, CosineSimilarity()),
    alignment=WatermanSmithBeyer(gap=ConstantGapCost(0), zero=0.1))

index = session.partition("sentence").index(metric, nlp)

In [13]:
import numpy as np
import json

class Logger:
    def __init__(self):
        self._data = []
        
    def __call__(self, hook, args):
        if isinstance(args, dict):
            c_args = dict()
            for k, v in args.items():
                if isinstance(v, np.ndarray):
                    c_args[k] = v.tolist()
                elif not callable(v):
                    c_args[k] = v
            self._data.append((hook, c_args))
        
    def write(self, path):
        with open(path, "w") as f:
            f.write(json.dumps(self._data, indent=4))

def debug(hook, args):
    print(hook)
    if hook == 'contextual_similarity_matrix':
        print(args)
    if hook == 'alignment/waterman-smith-beyer':
        print(args)
        
logger = Logger()

#index.find('less than two people', n=1, debug=logger)
index.find('they went to a new house', n=1, debug=logger)

FloatProgress(value=0.0, layout=Layout(width='100%'), max=1.0)

In [15]:
#logger.write("/Users/arbeit/Desktop/debug.json")

In [30]:
the_vectors = doc._contextual_embeddings["spacy/core_web_trf/3.0.0"].open()

In [38]:
session.documents[0]._contextual_embeddings['spacy/core_web_trf/3.0.0']._mask[:10]

array([ True,  True,  True,  True,  True, False,  True,  True,  True,
        True])

In [32]:
the_vectors.unmodified[:5, :10]

array([[-1.1437572 , -0.30409056,  0.86509454, -0.3199063 ,  1.0615821 ,
         0.8518285 ,  0.8528243 , -0.175081  , -0.36473972, -1.5114549 ],
       [-0.90024966, -0.7322849 , -0.06842124,  0.7308314 ,  1.216424  ,
         0.43938926,  0.8470165 ,  1.5145514 ,  1.4551923 , -0.8445735 ],
       [-0.56277   , -0.49145812, -0.16693702,  1.5647322 ,  2.4678462 ,
         1.9017525 , -0.59321785,  0.45290664, -0.14161064,  0.071206  ],
       [-0.37899685, -0.61451423,  0.47324866, -0.12926887,  1.2933862 ,
        -1.2349845 ,  0.9202763 , -0.1379444 , -1.2901033 , -0.41220453],
       [-0.16907969, -0.00849829, -0.05117168,  1.0145043 ,  0.6396689 ,
         1.594169  ,  1.2766489 ,  0.5413009 ,  0.87566674,  0.12214404]],
      dtype=float32)

In [33]:
the_vectors.unmodified[:5, :10]

array([[-0.7902165 ,  0.07575585,  1.1739827 , -0.59203947,  1.5813203 ,
         0.74327445,  0.5742742 , -0.41585118,  0.03771653, -1.5513849 ],
       [-0.69410986,  0.5108354 ,  0.02819886,  0.65676075,  2.4084206 ,
        -0.10979126,  0.82100856,  0.52209187,  1.8926276 , -0.86945325],
       [ 0.01929621, -0.5152243 ,  0.20008804,  1.3415744 ,  2.3048139 ,
         1.8081183 , -0.4049492 ,  0.09468511,  0.37712878, -0.5053382 ],
       [-0.17340645, -0.6229514 ,  0.58971316, -0.26212963,  2.6693375 ,
        -1.166319  ,  0.8589273 , -0.06591436, -1.6602695 , -0.707235  ],
       [ 0.14447287, -0.79191524, -0.7875701 , -0.3217711 ,  2.4091105 ,
         0.39358836, -0.5959423 , -0.78360283,  0.1599305 ,  0.6296562 ]],
      dtype=float32)

In [33]:
logger._data[0][1]['similarity']

[[0.03435264155268669],
 [0.20815441012382507],
 [0.12758424878120422],
 [0.040174320340156555],
 [0.08760717511177063],
 [0.07986205816268921],
 [0.077518992125988],
 [0.007615974172949791],
 [0.11193264275789261],
 [0.09225888550281525],
 [0.2719535827636719],
 [0.2404748797416687],
 [0.24498812854290009],
 [0.09525300562381744],
 [0.21946756541728973],
 [0.2555495798587799],
 [0.27379339933395386],
 [0.01946047507226467],
 [0.26449835300445557],
 [0.0787554681301117],
 [0.23769263923168182],
 [0.09757082164287567],
 [0.23840230703353882],
 [0.10723082721233368],
 [0.09300249814987183],
 [0.27191683650016785],
 [0.315754234790802],
 [0.1637067198753357],
 [0.04754708334803581],
 [0.17439493536949158],
 [0.24613851308822632],
 [0.3326062560081482],
 [0.18957677483558655],
 [0.04853488877415657],
 [0.28399544954299927],
 [0.2576620280742645],
 [0.08207577466964722],
 [0.1802542805671692],
 [0.522920548915863],
 [0.1728563755750656],
 [0.08114302158355713],
 [0.13426032662391663],
 [0.1

In [12]:
import json
with open("/Users/arbeit/Desktop/debug.json", "r") as f:
    data = json.loads(f.read())

In [27]:
np.array(data["a"])[0][:10]

array([ 1.11921835, -1.50500298, -0.47002545,  0.72598863,  1.25613248,
       -1.59027803, -0.11668806, -0.79666078,  0.65255964, -0.2121384 ])

In [26]:
np.array(data["b"])[-2][:10]

array([ 0.49695939, -1.57025933,  0.48457262, -0.32552609, -1.09349322,
       -1.80017662,  0.84969461, -0.1012421 ,  0.49484468, -0.32485196])

In [29]:
import numpy as np
import json
with open("/Users/arbeit/Desktop/debug2.json", "r") as f:
    data = json.loads(f.read())
np.array(data).shape

(2, 10)

In [29]:
with open("/Users/arbeit/Desktop/my_example_doc.json", "r") as f:
    data = json.loads(f.read())

In [31]:
sum(len(p["tokens"]) for p in data["partitions"])

837